# Craiglist car search

In [1]:
from bs4 import BeautifulSoup
import requests 
import time
import pandas as pd
import numpy as np

In [2]:
car_df = pd.read_excel('Car_search_terms.xlsx')

In [3]:
car_df.head()

,location,make,model
0,spokane,subaru,crosstrek
1,missoula,subaru,outback
2,saltlakecity,subaru,forester
3,boise,toyota,highlander
4,eastidaho,toyota,4runner


In [4]:
car_df['make_model'] = car_df['make'] + ' ' + car_df['model']
make_model_list = list(car_df['make_model'])
make_model_list = make_model_list[0:10]
make_model_list

['subaru crosstrek',
 'subaru outback',
 'subaru forester',
 'toyota highlander',
 'toyota 4runner',
 'toyota rav4',
 'honda crv',
 'honda pilot',
 'honda hrv',
 'honda h-rv']

In [5]:
location_list = list(car_df['location'])

In [6]:
from itertools import product
combos = pd.DataFrame(list(product(location_list, make_model_list)), columns=['location', 'make_model'])
combos

,location,make_model
0,spokane,subaru crosstrek
1,spokane,subaru outback
2,spokane,subaru forester
3,spokane,toyota highlander
4,spokane,toyota 4runner
...,...,...
145,sacramento,toyota rav4
146,sacramento,honda crv
147,sacramento,honda pilot
148,sacramento,honda hrv


In [7]:
combos['URL'] = "https://" + combos.location + ".craigslist.org/search/cta?purveyor=owner&query=" + combos.make_model

#Appending all links for specific cars urls
links= []
for url in combos['URL']:
    try:
        time.sleep(2)
        page = requests.get(url)
        page_soup = BeautifulSoup(page.content,'lxml')
        for link in page_soup.findAll('li',{'class':'cl-static-search-result'}):
            links.append(link.a['href'])
    except:
        print('error')
        print(url)

In [8]:
def post_id(page_soup):
    post_id = page_soup.findAll('p',{'class':'postinginfo'})[1].text.split()[2]
    return(post_id)

In [9]:
def title(page_soup):
    title = page_soup.findAll('span',{'class':'postingtitletext'})[0].text.replace('\n','')
    return(title)

In [10]:
def description(page_soup):
    description = page_soup.findAll('section',{'id':'postingbody'})[0].text.replace('\n',' ')
    return(description)

In [11]:
def price(page_soup):
    price = page_soup.findAll('span',{'class':"price"})[0].text
    return(price)

In [12]:
def milage(page_soup):
    for elements in page_soup.findAll('p', {'class' :'attrgroup'}):
        for item in elements.findAll('span'):
            item_text = item.text
            if ':' in item_text:
                field, value = item_text.split(':')
                if field == 'odometer':
                    return(value)

In [13]:
def crumb_area(page_soup):
    price = page_soup.findAll('span',{'class':"price"})[0].text
    return(price)

In [14]:
def year(page_soup):
    year = page_soup.find('div',class_='mapAndAttrs')
    year = year.find('p',class_='attrgroup').span.b.text[:4]
    return(year)

In [15]:
def Posting_date(page_soup):
    Posting_date = page_soup.findAll('p', {'class' :'postinginfo reveal'})[1].text.split()[1]
    return(Posting_date)

In [16]:
Final_Year = []
Final_Price = []
Final_Post_ID = []
Final_Title = []
Final_Description = []
Final_Millage = []
Final_Crumb_Area = []
Final_Posting_Date = []



for link in links:
    page = requests.get(link)
    page_soup = BeautifulSoup(page.content,'lxml')
    
    try:
        Final_Year.append(year(page_soup))
    except:
        Final_Year.append(0)
    
    try:
        Final_Price.append(price(page_soup))
    except:
        Final_Price.append('nan')
    
    try:
        Final_Post_ID.append(post_id(page_soup))
    except:
        Final_Post_ID.append(0)
    
    try:
        Final_Title.append(title(page_soup))
    except:
        Final_Title.append(0)
    
    try:
        Final_Description.append(description(page_soup))
    except:
        Final_Description.append(0)
    
    try:
        Final_Millage.append(milage(page_soup))
    except:
        Final_Millage.append(0)
    
    try:
        Final_Crumb_Area.append(crumb_area(page_soup))
    except:
        Final_Crumb_Area.append(0)
    
    try:
        Final_Posting_Date.append(Posting_date(page_soup))
    except:
        Final_Posting_Date.append(0)

In [17]:
links_dict = {'Year' : Final_Year, 'Price' : Final_Price, 'Post_ID' : Final_Post_ID, 'Title' : Final_Title, 'Description' : Final_Description, 'Millage' : Final_Millage, 'Crumb_Area' : Final_Crumb_Area, 'Posting_Date' : Final_Posting_Date}

Craiglist_Car_Information = pd.DataFrame(links_dict).set_index('Post_ID')

In [19]:
Craiglist_Car_Information.head()

,Year,Price,Title,Description,Millage,Crumb_Area,Posting_Date
Post_ID,,,,,,,
7717579278,0,"$16,900","2020 Subaru Crosstrek Premium 6spd - $16,900 (...",QR Code Link to This Post Rare 2020 Subaru...,None,"$16,900",2024-02-14
7716910499,0,"$17,000","2018 Subaru Crosstrek Limited AWD - $17,000 (S...",QR Code Link to This Post 2018 Subaru Cros...,None,"$17,000",2024-02-12
7716905887,0,"$16,500","2018 Subaru Crosstrek Limited AWD - $16,500 (S...",QR Code Link to This Post 2018 Subaru Cros...,None,"$16,500",2024-02-12
7715192534,0,"$18,000","2018 Subaru Crosstrek - Lifted, Off-Road Ready...",QR Code Link to This Post Selling my 2018 ...,None,"$18,000",2024-02-06
7716578554,0,"$7,950","2012 Subaru Outback AWD-92k miles- - $7,950 (s...",QR Code Link to This Post 2012 Subaru Outb...,None,"$7,950",2024-02-10


In [20]:
Craiglist_Car_Information.to_csv('craigscars.csv', index=False)